# Quick Start Guide

## cuOpt specific setup

- Activate Python virtual environment with CUDA runtime and GAMSPy installed
  - Install CUDA runtime via `pip install --extra-index-url=https://pypi.nvidia.com cuopt-cu12==25.5.* nvidia-cuda-runtime-cu12==12.8.* nvidia-nvjitlink-cu12`
  - Install GAMSPy via `pip install gamspy`
  - Install Jupyter notebooks via `pip install jupyter`
- Download latest cuOpt solver link zip archive (`cuopt-link-release.zip`) from the [releases page](https://github.com/GAMS-dev/cuoptlink-builder/releases).
- Extract archive into GAMS system directory for GAMSPy: `unzip -o cuopt-link-release.zip -d $(gamspy show base)`
- Run this notebook from a GUI or command line via `jupyter nbconvert --to notebook --execute --inplace trnsport_cuopt.ipynb`

The most relevant lines for the use of `cuopt` below are
```
gp.set_options({"SOLVER_VALIDATION": 0})
transport.solve(solver="cuopt", output=sys.stdout)
```

## Introduction

This quick start guide is designed to provide you with a concise overview of GAMSPy and its key features. By the end of this guide, you'll have a solid understanding of how to create basic mathematical models using GAMSPy. For more advanced features, we recommend exploring our comprehensive [user guide](https://gamspy.readthedocs.io/en/latest/user/index.html) and the extensive [model library](https://gamspy.readthedocs.io/en/latest/user/model_library.html).

While not mandatory, having a basic understanding of Python programming and familiarity with the [Pandas library](https://pandas.pydata.org/). will be helpful in following this tutorial.

## A Transportation Problem

In this guide, we'll delve into an example of the transportation problem. This classic scenario involves managing supplies from various plants to meet demands at multiple markets for a single commodity. Additionally, we have the unit costs associated with shipping the commodity from plants to markets. The fundamental economic question here is: 

> How can we optimize the shipment quantities between each plant and market to minimize the total transport cost?

The problem's algebraic representation typically takes the following format:

Indices (Sets):

- $i$ = plants
- $j$ = markets

Given Data (Parameters):

- $a_i$ = supply of commodity at plant $i$ (in cases)
- $b_j$ = demand for commodity at market $j$ (in cases)
- $c_{ij}$ = cost per unit of shipment between plant $i$ and market $j$

Decision Variables:

- $x_{ij}$ = amount of commodity to ship from plant $i$ to market $j$ where $x_{ij} \ge 0$ for all $i,j$

Constraints:

- Observe supply limit at plant $i$: $\sum_j x_{ij} \le a_i \: \forall i$
- Satisfy demand at market $j$: $\sum_i x_{ij} \ge b_j \: \forall j$
- Objective Function: Minimize $\sum_i \sum_j c_{ij} \cdot x_{ij}$


## Data

Before we dive into the optimization process, let's handle our data using the [Pandas library](https://pandas.pydata.org/). We'll begin by organizing the necessary information, which we will subsequently feed into our optimization model.

In [1]:
import pandas as pd

capacities = pd.DataFrame(
    [["seattle", 350], ["san-diego", 600]], columns=["city", "capacity"]
).set_index("city")

capacities

,capacity
city,
seattle,350
san-diego,600


In [2]:
demands = pd.DataFrame(
    [["new-york", 325], ["chicago", 300], ["topeka", 275]], columns=["city", "demand"]
).set_index("city")

demands

,demand
city,
new-york,325
chicago,300
topeka,275


In [3]:
distances = pd.DataFrame(
    [
        ["seattle", "new-york", 2.5],
        ["seattle", "chicago", 1.7],
        ["seattle", "topeka", 1.8],
        ["san-diego", "new-york", 2.5],
        ["san-diego", "chicago", 1.8],
        ["san-diego", "topeka", 1.4],
    ],
    columns=["from", "to", "distance"],
).set_index(["from", "to"])

distances

distance
from      to                
seattle   new-york       2.5
          chicago        1.7
          topeka         1.8
san-diego new-york       2.5
          chicago        1.8
          topeka         1.4

In [4]:
freight_cost = 90

## Symbol Declaration

In line with our systematic breakdown of the transportation problem into sets, parameters, variables, and constraints, we will adopt a similar approach to define the problem as a GAMSPy `Model`. To do so, it is essential to import the `gamspy` library initially.

In [5]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense

### Container

Before we proceed further, let's create a `Container` to encapsulate all the relevant information for our GAMSPy ``Model``. This ``Container`` acts as a centralized hub, gathering essential data, sets, parameters, variables, and constraints, providing a clear structure for our optimization problem.

In [6]:
m = Container()

### Sets

Sets serve as the fundamental building blocks of a GAMSPy ``Model``, directly corresponding to the indices in the algebraic representations of models. In our transportation problem context, we have defined the following indices:

- $i$ = plants
- $j$ = markets

For detailed guidance on using sets, please refer to the [set section](https://gamspy.readthedocs.io/en/latest/user/basics/set.html) of our user guide.

There a two ways to declare sets:

1. Separate declaration and data assignment
2. Combine declaration and data assignment

#### Separate declaration and data assignment


In [7]:
i = Set(container=m, name="i", description="plants")
i.setRecords(capacities.index)

#### Combine declaration and data assignment


In [8]:
j = Set(container=m, name="j", description="markets", records=demands.index)

The effect of using the above `Set` statements is that we declared two sets, namely $i$ and $j$. Additionally, we provided descriptions to elaborate on their meaning, enhancing the readability of our ``Model``. Lastly, we assigned members to the sets, establishing a clear connection between the abstract sets and their real-world counterparts.

$i$ = {Seattle, San Diego}

$j$ = {New York, Chicago, Topeka}

To verify the content of a set, you can use `<set name>.records`.

In [9]:
i.records

,uni,element_text
0,seattle,
1,san-diego,


In [10]:
j.records

,uni,element_text
0,new-york,
1,chicago,
2,topeka,


### Parameters

Declaring parameters involves using `Parameter`. Each parameter is assigned a name and a description.  Note that parameter $a_i$ is indexed by $i$. To accommodate these indices, we include the `domain` attribute, pointing to the corresponding set.

It is worth mentioning that, similar to sets, you have the flexibility to either combine or separate the declaration and data assignment steps. For convenience, we will proceed by combining the declaration and data assignment.


In [11]:
a = Parameter(
    container=m, 
    name="a",
    domain=i,
    description="supply of commodity at plant i (in cases)",
    records=capacities.reset_index(),
)
a.records

,city,value
0,seattle,350.0
1,san-diego,600.0


In [12]:
b = Parameter(
    container=m,
    name="b",
    domain=j,
    description="demand for commodity at market j (in cases)",
    records=demands.reset_index(),
)
b.records

,city,value
0,new-york,325.0
1,chicago,300.0
2,topeka,275.0


In [13]:
c = Parameter(
    container=m,
    name="c",
    domain=[i, j],
    description="cost per unit of shipment between plant i and market j",
)

The cost per unit of shipment between plant $i$ and market $j$ is derived from the distance between $i$ and $j$ and can be calculated as follows:

$c_{ij} = \frac{90 \cdot d_{ij}}{1000}$,

where $d_{ij}$ denotes the distance between $i$ and $j$.

We have two options to calculate $c_{ij}$ and assign the data to the GAMSPy parameter:

1. Python assignment - calculation in Python, e.g., using Pandas and `<parameter name>.setRecords()`
2. GAMSPy assignment - calculation in GAMSPy

#### Python Assignment

In [14]:
cost = freight_cost * distances / 1000
cost

distance
from      to                
seattle   new-york     0.225
          chicago      0.153
          topeka       0.162
san-diego new-york     0.225
          chicago      0.162
          topeka       0.126

In [15]:
c.setRecords(cost.reset_index())
c.records

,from,to,value
0,seattle,new-york,0.225
1,seattle,chicago,0.153
2,seattle,topeka,0.162
3,san-diego,new-york,0.225
4,san-diego,chicago,0.162
5,san-diego,topeka,0.126


#### GAMSPy Assignment

For the direct assignment we need to declare a new ``Parameter`` denoting the distances between $i$ and $j$.


In [16]:
d = Parameter(
    container=m,
    name="d",
    domain=[i, j],
    description="distance between plant i and market j",
    records=distances.reset_index(),
)
d.records

,from,to,value
0,seattle,new-york,2.5
1,seattle,chicago,1.7
2,seattle,topeka,1.8
3,san-diego,new-york,2.5
4,san-diego,chicago,1.8
5,san-diego,topeka,1.4


In [17]:
c[i, j] = freight_cost * d[i, j] / 1000
c.records

,i,j,value
0,seattle,new-york,0.225
1,seattle,chicago,0.153
2,seattle,topeka,0.162
3,san-diego,new-york,0.225
4,san-diego,chicago,0.162
5,san-diego,topeka,0.126


Further information on the usage of parameters can be found in our [parameter section](https://gamspy.readthedocs.io/en/latest/user/basics/parameter.html) of the user guide.

### Variables

GAMSPy variables are declared using `Variable`. Each ``Variable`` is assigned a name, a domain if necessary, a type, and, optionally, a description.

In [18]:
x = Variable(
    container=m,
    name="x",
    domain=[i, j],
    type="Positive",
    description="amount of commodity to ship from plant i to market j",
)

This statement results in the declaration of a shipment variable for each (i,j) pair.

More information on variables can be found in the [variable section](https://gamspy.readthedocs.io/en/latest/user/basics/variable.html) of our user guide.

## Equations
A GAMSPy ``Equation`` must be declared and defined in two separate statements. The format of the declaration is the same as for other GAMSPy symbols. First comes the keyword, `Equation` in this case, followed by the name, domain and text. The transportation problem has two constraints:

Supply: observe supply limit at plant $i$: $\sum_j x_{ij} \le a_i \: \forall i$

Demand: satisfy demand at market $j$: $\sum_i x_{ij} \ge b_j \: \forall j$

In [19]:
supply = Equation(
    container=m, name="supply", domain=i, description="observe supply limit at plant i"
)
demand = Equation(
    container=m, name="demand", domain=j, description="satisfy demand at market j"
)

The components of an ``Equation`` definition are:
1. The Python variable of the ``Equation`` being defined
2. The domain (optional)
3. Domain restricting conditions (optional)
4. A `=` sign
5. Left hand side expression
6. Relational operator (`==`, `<=`, `>=`)
7. The right hand side expression.

The ``Equation`` definition for the supply constraint of the transportation problem is implemented as follows:

In [20]:
supply[i] = Sum(j, x[i, j]) <= a[i]

Using the same logic as above, we can define the demand equation as follows:

In [21]:
demand[j] = Sum(i, x[i, j]) >= b[j]

More information on equations is given in the [equation section](https://gamspy.readthedocs.io/en/latest/user/basics/equation.html) of our user guide.

## Objective
The objective function of a GAMSPy ``Model`` does not require a separate ``Equation`` declaration. You can assign the objective expression to a Python variable or use it directly in the ``Model()`` statement of the [next section](#model).


In [22]:
obj = Sum((i, j), c[i, j] * x[i, j])

## Model

A GAMSPy `Model()` consolidates constraints, an objective function, a sense (minimize, maximize, and feasibility), and a problem type. It also possesses a name and is associated with a ``Container``.

To define our transportation problem as a GAMSPy ``Model``, we assign it to a Python variable, link it to our ``Container`` (populated with symbols and data), name it "transport", specify the equations, set the problem type as linear program (LP), specify the sense of the objective function (``Sense.MIN``), and point to the objective expression.

GAMSPy allows two alternatives to assign equations to a `Model`:
1. Using a list of equations,
2. Retrieving _all_ equations by calling `m.getEquations()`.

### Using a List of Equations
Using a list of equations is especially useful if you want to define multiple GAMSPy ``Model``s with a subset of the  equations in your ``Container``. For the transportation problem this can be done as follows:

In [23]:
transport = Model(
    m,
    name="transport",
    equations=[supply, demand],
    problem="LP",
    sense=Sense.MIN,
    objective=obj,
)

### Retrieving all Equations
Using `m.getEquations()` is especially convenient if you want to include all equations of your ``Container`` to be associated with your model. For the transportation problem this can be done as follows:

In [24]:
transport_2 = Model(
    m,
    name="transport2",
    equations=m.getEquations(),
    problem="LP",
    sense=Sense.MIN,
    objective=obj,
)

More information on the usage of a GAMSPy `Model` can be found in the [model section](https://gamspy.readthedocs.io/en/latest/user/basics/model.html) of our user guide.

## Solve

Upon defining the GAMSPy ``Model``, it's ready for being solved. The ``solve()`` statement triggers the generation of the specific model instance, creates suitable data structures for the solver, and invokes the solver. To view solver output in the console, the ``sys`` library can be used, passing the ``output=sys.stdout`` attribute to ``transport.solve()``.

In [25]:
import sys
import gamspy as gp

gp.set_options({"SOLVER_VALIDATION": 0})
transport.solve(solver="cuopt", output=sys.stdout)

--- Job _iryWq_R0QgaJ4suSj9QBhA.gms Start 07/14/25 15:42:59 50.1.0 12b75dde LEX-LEG x86 64bit/Linux


--- Applying:


    /home/gamsuser/andre/venvs/cu-opt/lib/python3.12/site-packages/gamspy_base/gmsprmun.txt


    /home/gamsuser/andre/venvs/cu-opt/lib/python3.12/site-packages/gamspy_base/gamsconfig.yaml


--- GAMS Parameters defined


...

--- _iryWq_R0QgaJ4suSj9QBhA.gms(201) 4 Mb


--- GDX File /tmp/tmpmdtq7_cf/_iryWq_R0QgaJ4suSj9QBhAout.gdx


*** Status: Normal completion


--- Job _iryWq_R0QgaJ4suSj9QBhA.gms Stop 07/14/25 15:42:59 elapsed 0:00:00.177


,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,153.675,6,7,LP,CUOPT,0.0


## Retrieving Results
### Variable Values
The values of the variables in the solution can be retrieved using using ``<variable name>.records``. The level specifies the shipment quantities $x_{ij}$. Other variable attributes are the marginal values, lower and upper bounds, and the variable's scaling factor.

In [26]:
x.records.set_index(["i", "j"])

level  marginal  lower  upper  scale
i         j                                             
seattle   new-york   50.0    0.0000    0.0    inf    1.0
          chicago   300.0    0.0000    0.0    inf    1.0
          topeka      0.0    0.0180    0.0    inf    1.0
san-diego new-york  275.0    0.0000    0.0    inf    1.0
          chicago     0.0    0.0045    0.0    inf    1.0
          topeka    275.0    0.0000    0.0    inf    1.0

### Objective Value
The optimal objective function value can be accessed by `<model name>.objective_value`.

In [27]:
transport.objective_value

153.675